In [7]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import imageio


In [8]:
def show3Dpose(channels, ax, lcolor="#3498db", rcolor="#e74c3c", add_labels=False): # blue, orange
  """
  Visualize a 3d skeleton

  Args
    channels: 96x1 vector. The pose to plot.
    ax: matplotlib 3d axis to draw on
    lcolor: color for left part of the body
    rcolor: color for right part of the body
    add_labels: whether to add coordinate labels
  Returns
    Nothing. Draws on ax.
  """

  assert channels.size == 32*3, "channels should have 96 entries, it has %d instead" % channels.size
  vals = np.reshape( channels, (32, -1) )

  I   = np.array([1,2,3,1,7,8,1, 13,14,15,14,18,19,14,26,27])-1 # start points
  J   = np.array([2,3,4,7,8,9,13,14,15,16,18,19,20,26,27,28])-1 # end points
  LR  = np.array([1,1,1,0,0,0,0, 0, 0, 0, 0, 0, 0, 1, 1, 1], dtype=bool)

  # Make connection matrix
  for i in np.arange( len(I) ):
    x, y, z = [np.array( [vals[I[i], j], vals[J[i], j]] ) for j in range(3)]
    ax.plot(x, y, z, lw=2, c=lcolor if LR[i] else rcolor)

  RADIUS = 750 # space around the subject
  xroot, yroot, zroot = vals[0,0], vals[0,1], vals[0,2]
  ax.set_xlim3d([-RADIUS+xroot, RADIUS+xroot])
  ax.set_zlim3d([-RADIUS+zroot, RADIUS+zroot])
  ax.set_ylim3d([-RADIUS+yroot, RADIUS+yroot])

  if add_labels:
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")

  # Get rid of the ticks and tick labels
  ax.set_xticks([])
  ax.set_yticks([])
  ax.set_zticks([])

  ax.get_xaxis().set_ticklabels([])
  ax.get_yaxis().set_ticklabels([])
  ax.set_zticklabels([])

In [14]:
import cdflib
import time
import os
cdf_file = cdflib.CDF("./data/h36m/S1/MyPoseFeatures/D3_Positions/Walking.cdf")
poses = cdf_file.varget("Pose").squeeze()

data = {}
data[0] = poses

filenames = []
images = []

for i in range(0, len(data[0]), 3):
    filepath = './temp/fig_'+ str(i) + '.png'
    filenames.append(filepath)
    if not os.path.exists(filepath):
        fig = plt.figure( figsize=(2, 2) )  
        ax = plt.subplot(projection='3d')
        plto = data[0][i]
        show3Dpose(plto, ax)
        plt.savefig('./temp/fig_'+ str(i))
        plt.close()

for filename in filenames:
    images.append(imageio.imread(filename))
    if os.path.exists(filename):
        os.remove(filename)

imageio.mimsave('./temp/movie.gif', images)


/var/folders/w0/pcnbsx215nb08cs5t4vk8jl40000gn/T/ipykernel_26798/3089134886.py:25: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(filename))
